In [1]:
import os
if "/content" in os.getcwd():
  !rm -rf WHere_codes/
  !git clone https://github.com/keenranger/WHere_codes
  os.chdir("/content/WHere_codes")
  !pip install -r requirements.txt
from PDR.modules.DataLoader import *
from PDR.modules.Walker import *

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import plotly.graph_objects as go

In [2]:
file_name = "200706_taereung"
pvloader = DataLoader("data/200706_taereung.csv", file_name)
pvloader.loader()
sensor_df = pvloader.sensor_df[["time", "accx", "accy", "accz", "gyrox", "gyroy",
                                "gyroz", "magx", "magy", "magz","rot0", "rot1", "rot2", "rot3", "game0", "game1", "game2", "game3"]]
sensor_df.head()

Data parsing... 
Data parsing done!


,time,accx,accy,accz,gyrox,gyroy,gyroz,magx,magy,magz,rot0,rot1,rot2,rot3,game0,game1,game2,game3
0,1593998250206,-1.228173,2.518592,8.980264,-0.471318,0.641334,0.184964,46.203240,7.184121,-27.968880,0.045117,0.078515,0.607698,0.788989,0.080057,0.042124,0.070451,0.993405
1,1593998250218,-1.168320,2.365370,8.810283,-0.545844,0.556423,0.221005,47.558006,5.865305,-27.080406,0.039910,0.080604,0.609043,0.788021,0.076584,0.046379,0.072218,0.993362
2,1593998250238,-1.065374,2.386916,9.056876,-0.661297,0.285199,0.242385,47.561188,5.140985,-27.416595,0.034628,0.081369,0.610723,0.786891,0.070885,0.050263,0.075020,0.993389
3,1593998250259,-0.816388,2.295941,10.117461,-0.460934,-0.082541,0.155643,49.214714,5.997710,-25.668380,0.027127,0.079198,0.613663,0.785118,0.065081,0.050826,0.077780,0.993545
4,1593998250278,-0.608101,2.384522,9.818199,-0.164664,-0.028174,0.053628,46.952293,4.995875,-27.585083,0.023903,0.075451,0.615497,0.784156,0.061200,0.049469,0.079818,0.993698


In [3]:
walker = Walker()
for row in tqdm(sensor_df.itertuples(), total=len(sensor_df)):
    walker.step(row[0], row[1], row[2:5], row[5:8], row[8:11], row[11:15], row[15:19])


100%|██████████| 5292/5292 [00:12<00:00, 438.73it/s]


In [4]:

displacement_df = pdr_to_displacement(walker.pdr_df)
position_df = displacement_df.cumsum()
walker.pdr_df

,idx,length,body,nav,rot,game,fusion
0,30.0,0.65,1.254476,1.254476,1.254476,1.254476,1.254476
1,56.0,0.65,1.147119,1.147119,1.147119,1.147119,1.147119
2,87.0,0.65,1.147164,1.147164,1.147164,1.147164,1.147164
3,109.0,0.65,1.314148,1.314148,1.314148,1.314148,1.314148
4,137.0,0.65,1.312988,1.312988,1.312988,1.312988,1.312988
...,...,...,...,...,...,...,...
212,5174.0,0.65,0.301224,-1.232306,-0.431737,-0.515129,-0.235283
213,5198.0,0.65,-0.258852,-1.820688,-0.955510,-1.105572,-0.825726
214,5228.0,0.65,-0.644187,-2.194659,-1.355151,-1.510496,-1.230650
215,5254.0,0.65,-0.734595,-2.322235,-1.486999,-1.608105,-1.328259


In [5]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=position_df['rot_x'], 
    y=position_df['rot_y'],
    name="rot"))
fig.add_trace(go.Scatter(
    x=position_df['game_x'], 
    y=position_df['game_y'],
    name="game"))
fig.add_trace(go.Scatter(
    x=position_df['nav_x'], 
    y=position_df['nav_y'],
    name="nav"))

for heading, position in zip(["nav", "rot", "game"], [["nav_x","nav_y"],["rot_x","rot_y"],["game_x", "game_y"]]):    
    corner_position = position_df[position][walker.corner_df[heading]]
    fig.add_trace(go.Scatter(
    x=corner_position[position[0]], 
    y=corner_position[position[1]],
    name= heading + "corner",
    mode = 'markers')
    )
fig.update_layout(
    title="PDR",
    width = 800,
    height = 800,
    yaxis = dict(
      scaleanchor = "x",
      scaleratio = 1,
    )
)
fig.show()

In [6]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=walker.headingcalc.heading_df['time'], y=np.rad2deg(walker.headingcalc.heading_df['rot']),name="rot"))
fig.add_trace(go.Scatter(x=walker.headingcalc.heading_df['time'], y=np.rad2deg(walker.headingcalc.heading_df['game']),name="game"))
fig.add_trace(go.Scatter(x=walker.headingcalc.heading_df['time'], y=np.rad2deg(walker.headingcalc.heading_df['nav']),name="nav"))
fig.update_layout(
    width = 800,
    height = 500
)
fig.show()

In [7]:
position_df[walker.corner_df["nav"]][["nav_x","nav_y"]]

,nav_x,nav_y
19,3.413419,12.383234
36,-6.776389,15.781305
78,2.065283,36.152781
110,9.477190,55.212082
119,5.008149,58.687860
177,19.070431,93.338808
203,28.051278,107.260199
212,32.820410,104.482683


In [8]:
fig = go.Figure()
mag_df = pvloader.sensor_df[['time','magx', 'magy', 'magz']]
mag_norm = np.sqrt(mag_df['magx']**2 + mag_df['magy']**2 + mag_df['magz']**2)
fig.add_trace(go.Scatter(x=mag_df['time'], y=mag_df['magx'], name="magx"))
fig.add_trace(go.Scatter(x=mag_df['time'], y=mag_df['magy'], name="magy"))
fig.add_trace(go.Scatter(x=mag_df['time'], y=mag_df['magz'], name="magz"))
fig.add_trace(go.Scatter(x=mag_df['time'], y=mag_norm, name="mag norm"))
fig.update_layout(
    width = 800,
    height = 500
)
fig.show()